In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

# Chlorophyll-a Concentration Prediction Project

## Objective

This project aims to predict chlorophyll-a concentrations in seawater using the CalCOFI dataset. Accurate prediction of chlorophyll-a levels will enable fishing companies to identify high-productivity zones, optimizing their operations.

## Scope

In this project, we will exclusively focus on testing and evaluating the performance of polynomial regression models. This will include:

- Data exploration and preparation.
- Implementation of polynomial regression with different polynomial degrees.
- Use of regularization techniques such as ridge, lasso, and elastic net.
- Application of cross-validation and grid search methods to find the best hyperparameters.

## Justification

The choice of polynomial regression as the main method is due to its ability to capture non-linear relationships between the predictor variables and the target variable (chlorophyll-a concentration). Additionally, using regularization will help prevent overfitting and improve model generalization.

## Methodology

1. **Exploratory Data Analysis (EDA)**: We will evaluate the distribution of variables, identify and handle missing values, and explore relationships between variables.
2. **Data Preparation**: Data cleaning and transformation, creation of polynomial features, and normalization.
3. **Model Training**: Training polynomial regression models with different degrees, applying regularization and cross-validation.
4. **Model Evaluation**: Assessing model performance using appropriate metrics such as Mean Squared Error (MSE) and R-squared (R²).

This systematic approach will allow us to identify the most suitable polynomial regression model for predicting chlorophyll-a concentration, thereby optimizing fishing operations.



# Business Case: Optimizing Fishing Operations through Chlorophyll-a Prediction

## Objective

Develop a predictive model to estimate chlorophyll-a concentrations in seawater to help fishing companies identify high-productivity zones. Accurate predictions of chlorophyll-a levels will enable more efficient allocation of fishing resources, reduce operational costs, and maximize fish yields.

## Problem Statement

Fishing companies face significant challenges in locating areas with high fish density. Current methods are often time-consuming and costly, leading to inefficiencies and higher operational costs. By predicting chlorophyll-a concentration, which is a key indicator of phytoplankton abundance, companies can better target their fishing efforts.

## Data Source

The California Cooperative Oceanic Fisheries Investigations (CalCOFI) dataset, which includes comprehensive measurements of oceanographic and environmental variables such as:

- Depth (Depthm)
- Water temperature (T_degC)
- Salinity (Salnty)
- Nutrient concentrations (e.g., nitrate NO3uM, phosphate PO4uM, silicate SiO3uM)

## Potential Predictors

To build a robust predictive model, the following variables could be considered as potential predictors:

- Environmental Variables: Depth, water temperature, salinity
- Chemical Variables: Concentrations of nitrate, phosphate, silicate, and dissolved oxygen
- Geographic and Temporal Variables: Latitude, longitude, time of year (month, season)

## Expected Outcome

By leveraging this model, fishing companies will be able to:

- Identify and prioritize high-yield fishing zones based on predicted chlorophyll-a levels.
- Optimize fishing routes and schedules, reducing fuel consumption and operational time.
- Increase fish yields by targeting areas with higher biological productivity.
- Enhance decision-making processes with data-driven insights into oceanographic conditions.

## Impact

Implementing this predictive model will lead to:

- Cost savings through more efficient resource allocation.
- Higher profitability due to increased fish yields.
- Reduced environmental impact by minimizing unnecessary fishing efforts.


In [9]:
# This datasets are not in the repo, you can download it from the kaggle webpage
# https://www.kaggle.com/datasets/sohier/calcofi
# Also we are going to be working with the variable description provided by the
# CalCOFI webpage: https://calcofi.org/data/oceanographic-data/bottle-database/

bottle = pd.read_csv("datasets/calcofi/bottle.csv")
cast = pd.read_csv("datasets/calcofi/cast.csv")

/tmp/ipykernel_2883/32580813.py:6: DtypeWarning: Columns (47,73) have mixed types. Specify dtype option on import or set low_memory=False.
  bottle = pd.read_csv("datasets/calcofi/bottle.csv")
/tmp/ipykernel_2883/32580813.py:7: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  cast = pd.read_csv("datasets/calcofi/cast.csv")


------

# Data Exploration

In this section, we will explore the dataset to understand its structure, identify missing values, and visualize relationships between variables.


In [10]:
# Let's work on a random sample of the dataset
# the parameter random_state should be used to get 
# same results in different executions of the code 

bottle = bottle.sample(n=10000, random_state=42)

First lets take a quick view over the objective variable 

In [19]:
print("\n\n\n#########.info()#############\n\n\n")
print(bottle["ChlorA"].info())
print("\n\n\n#########.sample(10)#############\n\n\n")
print(bottle["ChlorA"][bottle["ChlorA"].notna()].sample(10))
print("\n\n\n#########.describe()#############\n\n\n")
print(bottle["ChlorA"].describe())




#########.info()#############



<class 'pandas.core.series.Series'>
Index: 10000 entries, 246454 to 893
Series name: ChlorA
Non-Null Count  Dtype  
--------------  -----  
2602 non-null   float64
dtypes: float64(1)
memory usage: 156.2 KB
None



#########.sample(10)#############



841379    0.097
799866    0.690
550350    0.020
722984    0.170
519247    0.380
615546    0.100
834113    0.210
799322    1.860
526864    0.820
785642    2.960
Name: ChlorA, dtype: float64



#########.describe()#############



count    2602.000000
mean        0.473914
std         1.341153
min         0.000000
25%         0.050000
50%         0.150000
75%         0.390000
max        22.530000
Name: ChlorA, dtype: float64


1. The variable is numeric, stored in a float64 dtype and has a minimum of 0, maximum of 22.53 and a mean of 0.47. This kind of value is ideal for regression analysis!
2. If the mean is 0.47 and the std is 1.34, the maximum of 22,53 is clearly an outlier, we'll check it out later.
3. Just 26 percent of the rows contain values, that's a significant ammount of null values in the dataset. Before decide what to do with those null values, we should try to know more about the reason of those nulls.

Let's start looking at the big picture. We have two datasets: cast and bottle. 
**What is a bottle?** This is the answer from the CalCOFI website:

> **History of CalCOFI Seawater Sample Collection** <br> Prior to 1993, CalCOFI collected seawater samples using Niskin, Nansen, and “Wally” (in-house design by Walt Bryant & George Anderson) bottles with reversing thermometers. In 1990, CalCOFI began using a CTD-Rosette (pictured here underwater), and starting Aug 1993 (9308NH) CTD-Rosette sampling became the primary method of seawater collection. ![Example](https://calcofi.org/wp-content/uploads/2020/03/1904RL_sta93-50_CTDSurface-1024x770.jpg)


> - The Cast table contains metadata. This table includes date, time, latitude, longitude, weather, etc. for each CTD cast ever completed on a CalCOFI cruise. Each row is a unique cast, numbered sequentially/indexed by the “Cst_Cnt” column.
> - The Bottle table contains oceanographic data. This table includes oceanographic measurements for each bottle/sampling depth ever completed on a CalCOFI cruise. There are additional data code and precision columns describing the quality of each oceanographic measurement. Each row is a unique bottle/sampling depth, numbered sequentially/indexed by the “Btl_Cnt” column.
> - This table is much larger than the Cast table because each cast can have up to 24 bottles/sampling depths.
> - The Bottle table also has a “Cst_Cnt” column, which can be used as a unifier between the two tables.


Then cast seems to be the big device in the image which holds the bottles, also see that the bottles are numerated until 24 and then starts over in 1 as the description says. And bottle is each of those bottles in the cast. 